# Ubike api content description

http://data.taipei/opendata/datalist/datasetMeta?oid=8ef1626a-892a-4218-8344-f7ac46e1aa48

- sno：站點代號
- sna：場站名稱(中文)
- tot：場站總停車格
- sbi：場站目前車輛數量
- sarea：場站區域(中文)
- mday：資料更新時間
- lat：緯度
- lng：經度
- ar：地(中文)
- sareaen：場站區域(英文)
- snaen：場站名稱(英文)
- aren：地址(英文)
- bemp：空位數量
- act：全站禁用狀態

In [144]:
import requests
from math import sqrt
from urllib.parse import parse_qsl

In [135]:
lat, lng = (24.988454, 121.544298)

In [73]:
# get data from google geo api,

        
    
    def get_user_location(lat, lng):
        google_geo_api = 'http://maps.google.com/maps/api/geocode/json?latlng={},{}&language=zh-TW&sensor=true'.format(lat, lng)
        data = requests.get(google_geo_api)
        address_components = data.json()['results'][0]['address_components']
        print(address_components)
        user_location = 'undefined'
        for comp in address_components:
            if comp['types'][0] == 'administrative_area_level_1':
                user_location = comp['long_name']
                print(user_location)
                break
            elif comp['types'][0] == 'administrative_area_level_2':
                user_location = comp['long_name']
                print(user_location)
                break
        return user_location

    def num2str(value):
        str_num = str(value)
        if len(str_num) <= 4:
            return '0' * (4 - len(str_num)) + str_num
        return str_num

    def find_nearest_two_stop(lat, lng):
        """
        return two nearest stop with given lat and lng
        """

        # get data from data.taipei
        gz_data = requests.get('http://data.taipei/youbike')
        json_data = gz_data.json()

        # get all stop location
        ubike_locations = []
        for loc in json_data['retVal'].values():
            stop_info = dict(loc)
            ubike_locations.append({'lat': stop_info['lat'], 'lng': stop_info['lng']})

        # calculate all the distance between user location and stops
        dist_map = []
        for num, loc in enumerate(ubike_locations):
            stop_lat = float(loc['lat'])
            stop_lng = float(loc['lng'])
            dist = sqrt((lat - stop_lat) ** 2 + (lng - stop_lng) ** 2)
            dist_map.append({'dist': dist, 'stop_index': num2str(num + 1)}) # enumerate counts from 0

        sorted_dist_map = sorted(dist_map, key=lambda x: x['dist'])

        # the nearest two stop
        nearest_two_stop = sorted_dist_map[0:2]
        results = []
        for stop in nearest_two_stop:
            result = {}
            result['station'] = json_data['retVal'][stop['stop_index']]['sna']
            result['num_ubike'] = json_data['retVal'][stop['stop_index']]['sbi']
            results.append(result)

        return results


In [143]:
find_nearest_two_stop(lat,lng)

[{'num_ubike': '5', 'station': '考試院'}, {'num_ubike': '17', 'station': '捷運景美站'}]

In [76]:
lat, lng = (24.416703, 121.596021)

In [77]:
get_user_location(lat,lng)

[{'long_name': '平元林道', 'short_name': '平元林道', 'types': ['route']}, {'long_name': '金洋村', 'short_name': '金洋村', 'types': ['administrative_area_level_4', 'political']}, {'long_name': '南澳鄉', 'short_name': '南澳鄉', 'types': ['administrative_area_level_3', 'political']}, {'long_name': '宜蘭縣', 'short_name': '宜蘭縣', 'types': ['administrative_area_level_2', 'political']}, {'long_name': '台灣', 'short_name': 'TW', 'types': ['country', 'political']}, {'long_name': '272', 'short_name': '272', 'types': ['postal_code']}]
宜蘭縣


'宜蘭縣'